Primero importamos la librerias que usaremos

dejo esto aqui porque a veces no me deja instalar las librerias desde le cmd:

import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
install('pandas')


In [177]:
# Análisis de datos
import pandas as pd
import numpy as np
import random as rnd



# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


cargamos los datos



In [178]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')


## Analizamos el conjunto de datos

lo primero juntamos los datos, lo combino porque me gusta crear yo los conjuntos test y train

In [179]:
df_titatnic=pd.concat([train_df,test_df]).drop_duplicates('PassengerId').reset_index()


In [180]:
df_titatnic.head()

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [181]:
df_titatnic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        1309 non-null   int64  
 1   PassengerId  1309 non-null   int64  
 2   Survived     891 non-null    float64
 3   Pclass       1309 non-null   int64  
 4   Name         1309 non-null   object 
 5   Sex          1309 non-null   object 
 6   Age          1046 non-null   float64
 7   SibSp        1309 non-null   int64  
 8   Parch        1309 non-null   int64  
 9   Ticket       1309 non-null   object 
 10  Fare         1308 non-null   float64
 11  Cabin        295 non-null    object 
 12  Embarked     1307 non-null   object 
dtypes: float64(3), int64(5), object(5)
memory usage: 133.1+ KB


Parece ser que las variables categoriacas son cabin,embarked,ticket,Name,sex y el resto son numericas:





### Descripción

* PassengerId: ID del pasajero
* Survived: Supervivencia o no supervivencia (1 = Superviviente / 0 = No superviviente)
* Pclass: Clase del pasajero (1/2/3)
* Name: Nombre completo del pasajero
* Sex: Sexo del pasajero
* Age: Edad del pasajero
* SibSp: Hermanos/conyuges a bordo
* Parch: Padres/hijos a bordo
* Ticket: ID del ticket
* Fare: Tarifa
* Cabin: Camarote asignado
* Embarked: Puerta de embarque (C = Cherbourg, Q = Queenstown, S = Southampton)

In [182]:
df_titatnic.astype('object').describe().transpose()

,count,unique,top,freq
index,1309,891,0,2
PassengerId,1309,1309,1,1
Survived,891.0,2.0,0.0,549.0
Pclass,1309,3,3,709
Name,1309,1307,"Kelly, Mr. James",2
Sex,1309,2,male,843
Age,1046.0,98.0,24.0,47.0
SibSp,1309,7,0,891
Parch,1309,8,0,1002
Ticket,1309,929,CA. 2343,11


veamos la calidad de nuestro datos

In [183]:

columnastitanic=list(df_titatnic.columns)
columnastitanic

['index',
 'PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [184]:
for i in columnastitanic:
    s=df_titatnic[i].isnull().sum()
    print(i+':'+str(s))

index:0
PassengerId:0
Survived:418
Pclass:0
Name:0
Sex:0
Age:263
SibSp:0
Parch:0
Ticket:0
Fare:1
Cabin:1014
Embarked:2


voy a elimnar aquellos que no sabemos si sobreviven:

In [185]:
df_titatnic.count()

index          1309
PassengerId    1309
Survived        891
Pclass         1309
Name           1309
Sex            1309
Age            1046
SibSp          1309
Parch          1309
Ticket         1309
Fare           1308
Cabin           295
Embarked       1307
dtype: int64

In [186]:
titanic_fix = df_titatnic[df_titatnic['Survived'].notna()]
titanic_fix = titanic_fix[titanic_fix['Age'].notna()]

In [187]:
titanic_fix.count()

index          714
PassengerId    714
Survived       714
Pclass         714
Name           714
Sex            714
Age            714
SibSp          714
Parch          714
Ticket         714
Fare           714
Cabin          185
Embarked       712
dtype: int64

elimino la columna cabin ya que por fare se puede saber el nivel adquisitivo de una persona


In [188]:
titanic_fix['FareBand'] = pd.qcut(titanic_fix['Fare'], 4)
titanic_fix['AgeBand'] = pd.cut(titanic_fix['Age'], 5)

In [189]:
titanic_fix.to_csv('titanic_fix.csv',index=False,sep=";",decimal=",",encoding="utf-8-sig")

In [190]:
titanic_fix.drop(['Cabin','Name','Ticket','Embarked','FareBand','AgeBand'],
  axis='columns', inplace=True)
# edad la transformamos a numerica
titanic_fix['Sex'] = titanic_fix['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [191]:
titanic_fix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        714 non-null    int64  
 1   PassengerId  714 non-null    int64  
 2   Survived     714 non-null    float64
 3   Pclass       714 non-null    int64  
 4   Sex          714 non-null    int32  
 5   Age          714 non-null    float64
 6   SibSp        714 non-null    int64  
 7   Parch        714 non-null    int64  
 8   Fare         714 non-null    float64
dtypes: float64(3), int32(1), int64(5)
memory usage: 53.0 KB


Creo ahora mi conjunto test y train


In [192]:
randomvar = pd.DataFrame(np.random.randn(714, 2))

msk = np.random.rand(len(randomvar)) < 0.8

train_titanic = titanic_fix[msk]

test_titanic = titanic_fix[~msk]

vamos a predecir, ya que las visualizaciones las hare en power bi


# Modelar,Predecir


In [193]:
X_train = train_titanic.drop("Survived", axis=1)
Y_train = train_titanic["Survived"]
X_test  = test_titanic.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((589, 8), (589,), (125, 8))

In [194]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log


C:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


78.78

In [195]:

#vamos un resumen de los coeficientes
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
3,Sex,2.436725
1,Pclass,0.182405
7,Ticket,0.012344
4,Age,-0.027797
6,Parch,-0.101104
0,Survived,-0.181917
5,SibSp,-0.268465
2,Name,-0.690107
8,Fare,NaN
9,Cabin,NaN


Parece ser que las mujeres , com ose puede ver en la pelicula tienen mayor probabilidad de sobrevivir

In [196]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

63.67

In [197]:
#knn
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

81.15

In [198]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

79.12

In [199]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

67.4

In [200]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

C:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


45.33

In [201]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

61.29

In [202]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

100.0

In [203]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

100.0

resumen:

In [204]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,100.00
8,Decision Tree,100.00
1,KNN,81.15
4,Naive Bayes,79.12
2,Logistic Regression,78.78
5,Perceptron,67.40
0,Support Vector Machines,63.67
6,Stochastic Gradient Decent,61.29
7,Linear SVC,45.33


La verdad que tengo menos datos que el ejemplo dado en clase, pero veia poco sentido a tener datos con survived na y otros, te hare unas visualizaciones en power bi.